In [1]:
# | default_exp backend.analyzer

In [3]:
# | export
import json
import sys
import os
from typing import List, Dict
from rich.console import Console

from agentic.llm_factory import LLMClient
from agentic.schemas import Task, ExecutionPlan, AnalysisResult, ComplexityLevel, BuddyTool, ExecutionMode
from agentic.configs.prompts import AnalyzerPrompts

In [5]:
# | export
class AgentTaskAnalyzer:
    def __init__(self, model: str = None, base_url: str = None, api_key: str = None):
        self.llm_client = LLMClient(model=model, base_url=base_url, api_key=api_key)
        self.console = Console()

    def analyze(self, user_input: str) -> AnalysisResult:
        """Analyze input and create Buddy-executable task plan"""
        try:
            print("📊 Step 1: Assessing complexity...")
            # Step 1: Assess complexity
            complexity = self._assess_complexity(user_input)
            print(f"✅ Complexity: {complexity.value}")
            
            # Step 2: Create tasks (including framework selection task)
            if complexity == ComplexityLevel.SIMPLE:
                print("📋 Step 2: Creating simple task...")
                tasks = self._create_simple_task(user_input)
            else:
                print("📋 Step 2: Decomposing into tasks...")
                tasks = self._decompose_tasks(user_input, complexity)
            
            print(f"✅ Created {len(tasks)} tasks")
            
            print("🔄 Step 3: Creating execution plan...")
            # Step 3: Create execution plan
            execution_plan = self._create_execution_plan(tasks)
            
            # Step 4: Identify required Buddy tools
            buddy_tools = list(set([tool for task in tasks for tool in task.buddy_tools]))
            
            # Step 5: Define success criteria
            success_criteria = self._define_success_criteria(user_input, tasks)
            
            print("✅ Analysis complete!")
            
            return AnalysisResult(
                input_complexity=complexity,
                recommended_frameworks={},  # Will be determined during task execution
                tasks=tasks,
                execution_plan=execution_plan,
                buddy_tools_needed=buddy_tools,
                success_criteria=success_criteria
            )
            
        except Exception as e:
            print(f"❌ Analysis error: {e}")
            return self._create_fallback_result(user_input)

    def _assess_complexity(self, user_input: str) -> ComplexityLevel:
        """Assess input complexity using LLM analysis"""
        print("🔍 Analyzing complexity with LLM...")
        
        prompt = AnalyzerPrompts.complexity_analysis(user_input)
        
        try:
            response = self._call_llm(prompt)
            cleaned = self._clean_json(response)
            data = json.loads(cleaned)
            complexity_str = data.get("complexity", "moderate")
            print(f"🧠 LLM Assessment: {complexity_str} - {data.get('reasoning', '')}")
            return ComplexityLevel(complexity_str)
        except Exception as e:
            print(f"⚠️  LLM complexity analysis failed, using heuristic: {e}")
            # Fallback to word count heuristic
            word_count = len(user_input.split())
            if word_count < 20:
                return ComplexityLevel.SIMPLE
            elif word_count < 100:
                return ComplexityLevel.MODERATE
            else:
                return ComplexityLevel.COMPLEX

    def _select_frameworks(self, user_input: str, complexity: ComplexityLevel) -> Dict[str, str]:
        """Use debate agent to select optimal frameworks"""
        print("🗣️  Starting framework debate...")
        
        debate_prompt = AnalyzerPrompts.framework_selection(user_input, complexity.value)
        
        try:
            response = self._call_llm(debate_prompt)
            cleaned = self._clean_json(response)
            data = json.loads(cleaned)
            
            # Extract frameworks (exclude reasoning)
            frameworks = {k: v for k, v in data.items() if k != "reasoning"}
            
            if "reasoning" in data:
                print("💭 Framework reasoning:")
                for category, reason in data["reasoning"].items():
                    print(f"   {category}: {reason}")
            
            return frameworks
            
        except Exception as e:
            print(f"⚠️  Framework selection failed, using defaults: {e}")
            # Fallback frameworks based on complexity
            if complexity == ComplexityLevel.SIMPLE:
                return {"web_framework": "flask", "database": "sqlite"}
            elif complexity == ComplexityLevel.MODERATE:
                return {"web_framework": "fastapi", "database": "postgresql", "testing": "pytest"}
            else:
                return {
                    "web_framework": "fastapi", 
                    "database": "postgresql",
                    "ml_framework": "scikit-learn",
                    "container": "docker",
                    "orchestration": "kubernetes",
                    "ci_cd": "github-actions",
                    "monitoring": "prometheus",
                    "testing": "pytest"
                }

    def _create_simple_task(self, user_input: str) -> List[Task]:
        """Create optimized task for simple input"""
        user_lower = user_input.lower()
        
        # File search/grep operations
        if any(keyword in user_lower for keyword in ['list', 'find', 'search', 'files', 'containing', 'keyword', 'grep']):
            return [Task(
                id="T001",
                name="Search files with keyword",
                description=f"Execute file search request: {user_input}. Uses fs_read tool's grep functionality for efficient file searching.",
                complexity=ComplexityLevel.SIMPLE,
                buddy_tools=[BuddyTool.FS_READ],
                frameworks={},
                actions=[
                    {
                        "step": 1,
                        "action": "Search files using grep functionality",
                        "tool": "fs_read",
                        "purpose": "Find files containing specified keywords",
                        "sub_steps": ["Use grep mode to search across files", "Filter by file patterns if specified", "Return matching files with context"]
                    }
                ],
                success_criteria="Files containing the keyword are identified and listed",
                expected_outputs=["search_results"]
            )]
        
        # File/directory listing operations
        elif any(keyword in user_lower for keyword in ['list directory', 'show files', 'directory contents', 'ls']):
            return [Task(
                id="T001", 
                name="List directory contents",
                description=f"List directory contents: {user_input}. Uses fs_read tool's directory mode for efficient listing.",
                complexity=ComplexityLevel.SIMPLE,
                buddy_tools=[BuddyTool.FS_READ],
                frameworks={},
                actions=[
                    {
                        "step": 1,
                        "action": "List directory contents",
                        "tool": "fs_read",
                        "purpose": "Show files and directories in specified path",
                        "sub_steps": ["Use directory mode to list contents", "Apply depth limits if specified", "Show file details if requested"]
                    }
                ],
                success_criteria="Directory contents are listed successfully",
                expected_outputs=["directory_listing"]
            )]
        
        # File reading operations
        elif any(keyword in user_lower for keyword in ['read file', 'show file', 'cat', 'view file']):
            return [Task(
                id="T001",
                name="Read file contents", 
                description=f"Read file contents: {user_input}. Uses fs_read tool's line mode for efficient file reading.",
                complexity=ComplexityLevel.SIMPLE,
                buddy_tools=[BuddyTool.FS_READ],
                frameworks={},
                actions=[
                    {
                        "step": 1,
                        "action": "Read file contents",
                        "tool": "fs_read",
                        "purpose": "Display file contents with optional line ranges",
                        "sub_steps": ["Use line mode to read file", "Apply line range if specified", "Handle large files with chunking"]
                    }
                ],
                success_criteria="File contents are displayed successfully",
                expected_outputs=["file_contents"]
            )]
        
        # Code execution operations
        elif any(keyword in user_lower for keyword in ['run', 'execute', 'calculate', 'compute']):
            return [Task(
                id="T001",
                name="Execute code or calculation",
                description=f"Execute request: {user_input}. Uses code_interpreter for direct execution.",
                complexity=ComplexityLevel.SIMPLE,
                buddy_tools=[BuddyTool.CODE_INTERPRETER],
                frameworks={},
                actions=[
                    {
                        "step": 1,
                        "action": "Execute code or calculation",
                        "tool": "code_interpreter",
                        "purpose": "Run the requested computation or code",
                        "sub_steps": ["Parse the request", "Execute the code", "Return results"]
                    }
                ],
                success_criteria="Code executed successfully with expected output",
                expected_outputs=["execution_result"]
            )]
        
        # Default simple task for other requests
        else:
            return [Task(
                id="T001",
                name="Execute user request",
                description=f"Execute the user's request: {user_input}. This is a straightforward task that can be accomplished with basic tools.",
                complexity=ComplexityLevel.SIMPLE,
                buddy_tools=[BuddyTool.CODE_INTERPRETER, BuddyTool.INTROSPECT],
                frameworks={},
                actions=[
                    {
                        "step": 1,
                        "action": "Analyze and execute request",
                        "tool": "code_interpreter",
                        "purpose": "Understand and implement the solution",
                        "sub_steps": ["Parse user request", "Implement solution", "Return results"]
                    }
                ],
                success_criteria="Task completed successfully",
                expected_outputs=["solution_output"]
            )]

    def _decompose_tasks(self, user_input: str, complexity: ComplexityLevel) -> List[Task]:
        """Decompose complex input into detailed Buddy-executable tasks"""
        print("🔍 Analyzing requirements and breaking down into tasks...")
        
        # First, create framework selection task
        framework_task = Task(
            id="T000",
            name="Framework and Technology Selection",
            description="Use debate agent to evaluate and select optimal frameworks, libraries, and tools for the project. This ensures the best technology choices are made through structured analysis of alternatives.",
            complexity=ComplexityLevel.MODERATE,
            buddy_tools=[BuddyTool.DEBATE_AGENT, BuddyTool.INTROSPECT],
            frameworks={},
            actions=[
                {
                    "step": 1,
                    "action": "Define framework categories and options",
                    "tool": "introspect",
                    "purpose": "Identify all technology decisions needed",
                    "sub_steps": [
                        "List web frameworks: Flask vs FastAPI vs Django",
                        "List databases: PostgreSQL vs MySQL vs SQLite vs MongoDB", 
                        "List ML frameworks: scikit-learn vs TensorFlow vs PyTorch",
                        "List container tools: Docker vs Podman",
                        "List orchestration: Kubernetes vs Docker Swarm",
                        "List CI/CD: GitHub Actions vs GitLab CI vs Jenkins",
                        "List monitoring: Prometheus vs DataDog vs New Relic"
                    ]
                },
                {
                    "step": 2,
                    "action": "Conduct framework debate",
                    "tool": "debate_agent",
                    "purpose": "Evaluate pros/cons of each option and select best choices",
                    "sub_steps": [
                        "Present arguments for each framework option",
                        "Consider project requirements and constraints",
                        "Evaluate performance, scalability, and maintainability",
                        "Make final selections with reasoning"
                    ]
                },
                {
                    "step": 3,
                    "action": "Validate framework selections",
                    "tool": "introspect",
                    "purpose": "Ensure selected frameworks are compatible and optimal",
                    "sub_steps": [
                        "Check framework compatibility",
                        "Verify licensing and support",
                        "Confirm alignment with project goals"
                    ]
                }
            ],
            success_criteria="Framework selections made with clear reasoning and compatibility verified",
            expected_outputs=["framework_decisions.json", "technology_rationale.md"]
        )
        
        prompt = AnalyzerPrompts.task_decomposition(user_input)
        
        try:
            response = self._call_llm(prompt)
            cleaned = self._clean_json(response)
            task_data = json.loads(cleaned)
            
            tasks = [framework_task]  # Start with framework selection task
            for data in task_data:
                task = Task(
                    id=data.get("id", f"T{len(tasks):03d}"),
                    name=data.get("name", "Unnamed task"),
                    description=data.get("description", ""),
                    complexity=ComplexityLevel(data.get("complexity", "moderate")),
                    dependencies=data.get("dependencies", []),
                    buddy_tools=[BuddyTool(tool) for tool in data.get("buddy_tools", ["fs_write"])],
                    execution_mode=ExecutionMode(data.get("execution_mode", "sequential")),
                    frameworks=data.get("frameworks", {}),
                    actions=data.get("actions", []),
                    success_criteria=data.get("success_criteria", "Task completed"),
                    expected_outputs=data.get("expected_outputs", [])
                )
                tasks.append(task)
            
            return tasks if len(tasks) > 1 else [framework_task] + self._create_simple_task(user_input)
            
        except Exception as e:
            print(f"❌ Task decomposition failed: {e}")
            return [framework_task] + self._create_simple_task(user_input)

    def _create_execution_plan(self, tasks: List[Task]) -> ExecutionPlan:
        """Create execution plan for Buddy"""
        # Build dependency graph
        task_map = {task.id: task for task in tasks}
        
        # Find execution phases
        phases = []
        remaining = tasks.copy()
        
        while remaining:
            # Find tasks with no unmet dependencies
            ready = []
            for task in remaining:
                deps_met = all(dep not in [t.id for t in remaining] for dep in task.dependencies)
                if deps_met:
                    ready.append(task)
            
            if not ready:
                ready = [remaining[0]]  # Break cycles
            
            # Separate parallel and sequential tasks
            parallel = [t for t in ready if t.execution_mode == ExecutionMode.PARALLEL]
            sequential = [t for t in ready if t.execution_mode == ExecutionMode.SEQUENTIAL]
            
            if parallel:
                phases.append([t.id for t in parallel])
            
            for task in sequential:
                phases.append([task.id])
            
            # Remove processed tasks
            for task in ready:
                remaining.remove(task)
        
        # Find parallel opportunities
        parallel_groups = [phase for phase in phases if len(phase) > 1]
        
        # Find critical path (simplified)
        critical_path = [phase[0] for phase in phases]
        
        # Count total actions
        total_actions = sum(len(task.actions) for task in tasks)
        
        return ExecutionPlan(
            sequential_phases=phases,
            parallel_groups=parallel_groups,
            critical_path=critical_path,
            total_actions=total_actions
        )

    def _define_success_criteria(self, user_input: str, tasks: List[Task]) -> List[str]:
        """Define project success criteria"""
        criteria = ["All tasks completed successfully"]
        
        # Add task-specific criteria
        unique_criteria = set()
        for task in tasks:
            if task.success_criteria and task.success_criteria not in unique_criteria:
                unique_criteria.add(task.success_criteria)
        
        criteria.extend(list(unique_criteria)[:3])  # Max 3 additional criteria
        return criteria

    def _call_llm(self, prompt: str) -> str:
        """Call LLM using llm_factory with streaming"""
        try:
            messages = [{"role": "user", "content": prompt}]
            response = self.llm_client.create_completion(messages=messages, tools=None, stream=True)
            
            # Use llm_factory's streaming handler
            result = self.llm_client.handle_streaming_response(response, self.console)
            return result.get("content", "")
            
        except Exception as e:
            print(f"\n❌ LLM call failed: {e}")
            return ""

    def _clean_json(self, response: str) -> str:
        """Extract clean JSON from response"""
        response = response.strip()
        
        # Remove thinking tags
        if "<think>" in response and "</think>" in response:
            start = response.find("</think>") + 8
            response = response[start:].strip()
        
        # Extract from code blocks
        if "```json" in response:
            start = response.find("```json") + 7
            end = response.find("```", start)
            response = response[start:end].strip()
        elif "```" in response:
            start = response.find("```") + 3
            end = response.find("```", start)
            response = response[start:end].strip()
        
        # Find JSON boundaries
        if response.startswith("["):
            bracket_count = 0
            for i, char in enumerate(response):
                if char == "[":
                    bracket_count += 1
                elif char == "]":
                    bracket_count -= 1
                    if bracket_count == 0:
                        return response[:i+1]
        
        return response

    def _create_fallback_result(self, user_input: str) -> AnalysisResult:
        """Create fallback result when analysis fails"""
        simple_task = Task(
            id="T001",
            name="Execute user request",
            description=f"Execute the user's request: {user_input}. This fallback task handles the request when detailed analysis fails.",
            complexity=ComplexityLevel.SIMPLE,
            buddy_tools=[BuddyTool.CODE_INTERPRETER, BuddyTool.INTROSPECT],
            frameworks={},
            actions=[
                {
                    "step": 1,
                    "action": "Analyze request",
                    "tool": "introspect",
                    "purpose": "Understand requirements",
                    "sub_steps": ["Parse request", "Identify needs"]
                },
                {
                    "step": 2,
                    "action": "Execute task",
                    "tool": "code_interpreter",
                    "purpose": "Implement solution",
                    "sub_steps": ["Create solution", "Test output"]
                }
            ],
            success_criteria="Request completed successfully"
        )
        
        return AnalysisResult(
            input_complexity=ComplexityLevel.SIMPLE,
            recommended_frameworks={},
            tasks=[simple_task],
            execution_plan=ExecutionPlan(
                sequential_phases=[["T001"]],
                critical_path=["T001"],
                total_actions=2
            ),
            buddy_tools_needed=[BuddyTool.CODE_INTERPRETER, BuddyTool.INTROSPECT],
            success_criteria=["Request completed successfully"]
        )

    def analyze_to_json(self, user_input: str) -> dict:
        """Analyze input and return JSON directly"""
        result = self.analyze(user_input)
        return self.to_json(result)

    def to_json(self, result: AnalysisResult) -> dict:
        """Convert analysis result to JSON format"""
        return {
            "input_complexity": result.input_complexity.value,
            "total_tasks": len(result.tasks),
            "recommended_frameworks": result.recommended_frameworks,
            "buddy_tools_needed": [tool.value for tool in result.buddy_tools_needed],
            "total_actions": result.execution_plan.total_actions,
            "tasks": [
                {
                    "id": task.id,
                    "name": task.name,
                    "description": task.description,
                    "complexity": task.complexity.value,
                    "dependencies": task.dependencies,
                    "buddy_tools": [tool.value for tool in task.buddy_tools],
                    "execution_mode": task.execution_mode.value,
                    "frameworks": task.frameworks,
                    "actions": task.actions,
                    "success_criteria": task.success_criteria,
                    "expected_outputs": task.expected_outputs
                }
                for task in result.tasks
            ],
            "execution_plan": {
                "sequential_phases": result.execution_plan.sequential_phases,
                "parallel_groups": result.execution_plan.parallel_groups,
                "critical_path": result.execution_plan.critical_path,
                "total_actions": result.execution_plan.total_actions
            },
            "success_criteria": result.success_criteria
        }

    def display_result(self, result: AnalysisResult) -> None:
        """Display analysis result in beautiful format"""
        from rich.panel import Panel
        from rich.table import Table
        from rich.columns import Columns
        from rich.text import Text
        
        # Header
        header = Text("🤖 BUDDY TASK ANALYSIS", style="bold cyan")
        self.console.print(Panel(header, style="bright_blue"))
        
        # Overview metrics
        overview_table = Table(show_header=False, box=None, padding=(0, 2))
        overview_table.add_column(style="bold yellow")
        overview_table.add_column(style="bright_white")
        
        overview_table.add_row("📊 Complexity:", result.input_complexity.value.upper())
        overview_table.add_row("🎯 Total Tasks:", str(len(result.tasks)))
        overview_table.add_row("⚡ Total Actions:", str(result.execution_plan.total_actions))
        overview_table.add_row("🛠️  Buddy Tools:", ", ".join([tool.value for tool in result.buddy_tools_needed]))
        
        if result.recommended_frameworks:
            frameworks_str = ", ".join([f"{k}:{v}" for k, v in result.recommended_frameworks.items()])
            overview_table.add_row("🔧 Frameworks:", frameworks_str)
        
        self.console.print(Panel(overview_table, title="📈 Overview", style="green"))
        
        # Task breakdown
        task_table = Table(show_header=True, header_style="bold magenta")
        task_table.add_column("ID", style="cyan", width=6)
        task_table.add_column("Task Name", style="bright_white", min_width=30)
        task_table.add_column("Tools", style="yellow", width=20)
        task_table.add_column("Actions", style="green", width=8)
        task_table.add_column("Dependencies", style="blue", width=12)
        
        for task in result.tasks:
            tools_str = ", ".join([tool.value for tool in task.buddy_tools])
            deps_str = ", ".join(task.dependencies) if task.dependencies else "None"
            
            # Highlight framework selection task
            if task.id == "T000":
                task_name = f"🎯 {task.name}"
            else:
                task_name = task.name
                
            task_table.add_row(
                task.id,
                task_name,
                tools_str,
                str(len(task.actions)),
                deps_str
            )
        
        self.console.print(Panel(task_table, title="📋 Task Breakdown", style="blue"))
        
        # Execution plan
        plan_text = Text()
        for i, phase in enumerate(result.execution_plan.sequential_phases, 1):
            mode = "🔄 PARALLEL" if len(phase) > 1 else "➡️  SEQUENTIAL"
            tasks_str = " + ".join(phase) if len(phase) > 1 else phase[0]
            plan_text.append(f"Phase {i}: {tasks_str} ({mode})\n", style="bright_white")
        
        self.console.print(Panel(plan_text, title="🔄 Execution Plan", style="yellow"))
        
        # Success criteria
        criteria_text = Text()
        for criteria in result.success_criteria:
            criteria_text.append(f"✅ {criteria}\n", style="green")
        
        self.console.print(Panel(criteria_text, title="🎯 Success Criteria", style="green"))
        
        # JSON output (collapsible)
        json_str = json.dumps(self.to_json(result), indent=2)
        self.console.print(Panel(f"[dim]{json_str}[/dim]", title="📄 JSON Output", style="dim", expand=False))
